# SI Opportunity Scoring — Recommended Modeling Notebook (v9)
**Champion model:** Weighted rule scorecard (data-driven, constrained, explainable)  
**Challenger model:** Calibrated Logistic Regression (shadow mode until pilot labels)  
**Evaluation:** lift/precision at top buckets + calibration + governance

**Last updated:** 2026-02-27

---

## Business problem
We want to find **client IDs with `si_offering = 0`** (not currently in an SI offering) that are **most likely to be interested** in an SI offering.

## What makes this tricky
We do *not* have a perfect “interest” label. Our initial benchmark label is a **proxy**:
- `si_offering = 1` if the client appears in an offering whose name contains the token **SI** (from `OFFERING_NAME`)

This label is useful for comparing methods, but it can be biased by commercial processes.  
Therefore, the optimal strategy is to:
1) start with a **constrained scorecard** (robust to label noise, easy governance)  
2) run a **pilot** to collect true outcomes (responses / adoption / pipeline)  
3) then upgrade to ML / uplift modeling using real labels.

---

## Modeling logic (recommended)
### Branching by MiFID
- If `MIFID = 0` -> score uses **only** `SI_CONSIDERATION`
- If `MIFID = 1` -> score = **alpha * SI + (1-alpha) * Confirmations**
  - SI is the anchor (trusted explicit signal)
  - Confirmations come from: **SFDR opportunity**, **PAI + topics**, **Taxonomy (A1/A2/A3)**

### Why alpha (SI vs confirmations)?
- If alpha too high -> ignore useful confirmation evidence
- If alpha too low -> over-infer preferences -> irrelevant outreach risk

We start with **alpha=0.80** (stakeholder-friendly) and learn an improved alpha from data **within bounds** (e.g., 0.60–0.90).

---

## Methods compared (same train/validation split)
1) **Fixed rule**: alpha fixed (0.80), confirmation weights fixed  
2) **Weighted rule (champion)**: learn confirmation weights + learn alpha (bounded)  
3) **ML (challenger)**: Calibrated Logistic Regression with MiFID interactions

Success is primarily about **top-bucket ROI**, so we emphasize:
- **Lift / Precision@K** (top 10%, 20%)
- **Decile lift curves**
- **Calibration** (probability reliability for bucket thresholds)

---
## 0) Setup

In [ ]:
import numpy as np
import pandas as pd
from dataclasses import dataclass
from pathlib import Path

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss

import matplotlib.pyplot as plt
pd.set_option("display.max_columns", 200)

---
## 1) Load raw data + shuffle rows

In [ ]:
DATA_PATH = Path("data.csv")  # <-- change to your real file path

def make_synthetic_data(n=9000, seed=42):
    rng = np.random.default_rng(seed)
    return pd.DataFrame({
        "ID": rng.integers(1, n//2 + 1, size=n),
        "IO_TYPE": rng.choice(["normal", "zombie"], size=n, p=[0.97, 0.03]),
        "LIFE_CYCLE": rng.choice(["open", "closed"], size=n, p=[0.9, 0.1]),
        "OFFERING_NAME": rng.choice(
            ["Core", "Standard", "ESG Plus", "SI Focus", "Core SI", "Income", "SI Sustainable", None],
            size=n, p=[0.23,0.23,0.14,0.14,0.08,0.07,0.06,0.05]
        ),
        "SI_CONSIDERATION_CD": rng.choice(["S1","S2","S3", None], size=n, p=[0.35,0.35,0.2,0.1]),
        "SFDR_PREF": rng.choice(["F1","F2","F3", None], size=n, p=[0.4,0.35,0.2,0.05]),
        "SFDR_ACTUAL": rng.choice(["F1","F2","F3", None], size=n, p=[0.45,0.35,0.15,0.05]),
        "PAI_PREF": rng.choice(["PAI Selected", None], size=n, p=[0.3,0.7]),
        "MIFID": rng.choice(["Yes","No", None], size=n, p=[0.55,0.4,0.05]),
        "TAXONOMYPREF": rng.choice(["A1","A2","A3", None], size=n, p=[0.5,0.35,0.1,0.05]),
        "GHG": rng.choice(["Yes","No","--", None], size=n, p=[0.25,0.65,0.05,0.05]),
        "Biodiversity": rng.choice(["Yes","No","--", None], size=n, p=[0.2,0.7,0.05,0.05]),
        "Water": rng.choice(["Yes","No","--", None], size=n, p=[0.22,0.68,0.05,0.05]),
        "Waste": rng.choice(["Yes","No","--", None], size=n, p=[0.18,0.72,0.05,0.05]),
        "Social": rng.choice(["Yes","No","--", None], size=n, p=[0.28,0.62,0.05,0.05]),
    })

if DATA_PATH.exists():
    df_raw = pd.read_csv(DATA_PATH)
    print(f"Loaded: {DATA_PATH}  shape={df_raw.shape}")
else:
    df_raw = make_synthetic_data()
    print("DATA_PATH not found; using synthetic demo dataset.")
    print(f"shape={df_raw.shape}")

# Shuffle to avoid ordering artifacts
df_raw = df_raw.sample(frac=1, random_state=42).reset_index(drop=True)
df_raw.head()

---
## 2) Cleaning & filtering (ONLY)

This step is **only** hygiene: filtering + missing normalization. No category->number mappings here.

In [ ]:
REQUIRED_RAW = [
    "ID","IO_TYPE","LIFE_CYCLE","OFFERING_NAME",
    "SI_CONSIDERATION_CD","SFDR_PREF","SFDR_ACTUAL","PAI_PREF","MIFID","TAXONOMYPREF",
    "GHG","Biodiversity","Water","Waste","Social"
]
missing_cols = [c for c in REQUIRED_RAW if c not in df_raw.columns]
if missing_cols:
    raise ValueError(f"Missing required raw columns: {missing_cols}")

def clean_filter_only(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # Trim strings
    for c in df.columns:
        if df[c].dtype == "object":
            df[c] = df[c].apply(lambda x: x.strip() if isinstance(x, str) else x)
    # Normalize placeholder missing
    df = df.replace({"--": np.nan, "": np.nan})

    before = len(df)
    df = df[df["IO_TYPE"].fillna("").str.lower() != "zombie"]
    after_zombie = len(df)
    df = df[df["LIFE_CYCLE"].fillna("").str.lower() == "open"]
    after_open = len(df)

    df.attrs["cleaning_summary"] = {
        "before": before,
        "after_remove_zombie": after_zombie,
        "after_keep_open": after_open,
        "removed_zombie": before - after_zombie,
        "removed_closed": after_zombie - after_open
    }
    return df

df_clean = clean_filter_only(df_raw)
pd.DataFrame([df_clean.attrs["cleaning_summary"]])

In [ ]:
# Cleaning impact chart
s = df_clean.attrs["cleaning_summary"]
plt.figure(figsize=(8,4))
plt.bar(["Before", "After zombie", "After open"], [s["before"], s["after_remove_zombie"], s["after_keep_open"]])
plt.title("Cleaning impact: rows remaining after filters")
plt.ylabel("Rows")
plt.grid(True, axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

# Missingness snapshot (raw values)
key_cols = ["SI_CONSIDERATION_CD","SFDR_PREF","SFDR_ACTUAL","TAXONOMYPREF","MIFID","PAI_PREF",
            "GHG","Biodiversity","Water","Waste","Social","OFFERING_NAME"]
miss = df_clean[key_cols].isna().mean().sort_values(ascending=False)
display(miss.to_frame("missing_rate").head(12))

plt.figure(figsize=(10,4))
plt.bar(miss.index[:12], miss.values[:12])
plt.title("Top missing rates after cleaning (raw values)")
plt.ylabel("Missing rate")
plt.xticks(rotation=45, ha="right")
plt.grid(True, axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

---
## 3) Target derivation + ID-level aggregation

We recommend **clients (IDs)**, so we aggregate to one row per ID.

**Proxy label:** `si_offering = 1` if any row for the client has `OFFERING_NAME` containing token `SI`.

In [ ]:
df = df_clean.copy()
df["si_offering_row"] = df["OFFERING_NAME"].astype(str).str.contains(r"\bSI\b", case=False, na=False).astype(int)

def mode_or_first(s: pd.Series):
    s2 = s.dropna()
    if len(s2) == 0:
        return np.nan
    m = s2.mode()
    return m.iloc[0] if len(m) else s2.iloc[0]

agg = {
    "OFFERING_NAME": mode_or_first,
    "SI_CONSIDERATION_CD": mode_or_first,
    "SFDR_PREF": mode_or_first,
    "SFDR_ACTUAL": mode_or_first,
    "PAI_PREF": mode_or_first,
    "MIFID": mode_or_first,
    "TAXONOMYPREF": mode_or_first,
    "GHG": mode_or_first,
    "Biodiversity": mode_or_first,
    "Water": mode_or_first,
    "Waste": mode_or_first,
    "Social": mode_or_first,
    "si_offering_row": "max",
}
df_id = df.groupby("ID", as_index=False).agg(agg).rename(columns={"si_offering_row":"si_offering"})
df_id["si_offering"] = df_id["si_offering"].astype(int)

sizes = pd.DataFrame({
    "level": ["row-level (cleaned)", "ID-level"],
    "rows": [len(df), len(df_id)],
    "si_offering_rate": [df["si_offering_row"].mean(), df_id["si_offering"].mean()]
})
display(sizes)

plt.figure(figsize=(7,4))
plt.bar(sizes["level"], sizes["rows"])
plt.title("Size: cleaned rows vs ID-level")
plt.ylabel("Count")
plt.grid(True, axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,4))
plt.bar(sizes["level"], sizes["si_offering_rate"])
plt.title("Proxy label prevalence: si_offering rate")
plt.ylabel("Rate")
plt.ylim(0,1)
plt.grid(True, axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

df_id.head()

---
## 4) Feature mappings

Stakeholder-friendly mapping table for key engineered features (kept intentionally small).

In [ ]:
feature_map = pd.DataFrame([
    ["SI_CONSIDERATION_CD", "S1/S2/S3", "si_norm", "S1=0, S2=0.5, S3=1"],
    ["MIFID", "Yes/No", "MIFID", "Yes=1, No/NA=0"],
    ["SFDR_PREF / SFDR_ACTUAL", "F1/F2/F3", "sfdr_gap, sfdr_norm", "sfdr_gap=clip(pref-actual,-2,2); sfdr_norm=max(gap,0)/2"],
    ["PAI_PREF + topics", "PAI Selected + ESG topics", "pai_block", "0 if no PAI else 0.5 + 0.5*topics_norm"],
    ["TAXONOMYPREF", "A1/A2/A3", "tax_norm", "A1=0, A2=0.5, A3=1 (scaled)"],
], columns=["Raw field(s)", "Raw values", "Engineered feature", "Definition"])
display(feature_map)

---
## 5) Encoding + engineered signals

Numeric encoding happens here.

### SFDR opportunity
- `sfdr_gap = clip(pref - actual, -2, 2)`
- `sfdr_norm = max(sfdr_gap, 0) / 2` (range 0..1)

This ensures only **pref > actual** drives opportunity.

In [ ]:
MAP_SI = {"S1":1, "S2":2, "S3":3}
MAP_SFDR = {"F1":1, "F2":2, "F3":3}
MAP_TAX = {"A1":1, "A2":2, "A3":3}

def yes_to_1(x):
    return 1 if isinstance(x, str) and x.strip().lower() == "yes" else 0

def encode(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # Topics Yes/No -> 1/0
    for c in ["GHG","Biodiversity","Water","Waste","Social"]:
        df[c] = df[c].apply(yes_to_1).astype(int)

    # MIFID Yes/No -> 1/0
    df["MIFID"] = df["MIFID"].apply(yes_to_1).astype(int)

    # PAI flag
    df["PAI_PREF"] = (df["PAI_PREF"].astype(str).str.lower() == "pai selected").astype(int)

    # Ordinals (default to lowest tier when missing)
    df["SI_CONSIDERATION_num"] = df["SI_CONSIDERATION_CD"].map(MAP_SI).fillna(1).astype(int)
    df["SFDR_PREF_num"] = df["SFDR_PREF"].map(MAP_SFDR).fillna(1).astype(int)
    df["SFDR_ACTUAL_num"] = df["SFDR_ACTUAL"].map(MAP_SFDR).fillna(1).astype(int)
    df["TAXONOMYPREF_num"] = df["TAXONOMYPREF"].map(MAP_TAX).fillna(1).astype(int)

    # Engineered
    df["sfdr_gap"] = np.clip(df["SFDR_PREF_num"] - df["SFDR_ACTUAL_num"], -2, 2)
    df["sfdr_norm"] = np.maximum(df["sfdr_gap"], 0) / 2.0   # 0..1 only (opportunity side)

    topic_cols = ["GHG","Biodiversity","Water","Waste","Social"]
    df["esg_topics_yes_cnt"] = df[topic_cols].sum(axis=1)
    df["topics_norm"] = df["esg_topics_yes_cnt"] / len(topic_cols)

    df["pai_block"] = np.where(df["PAI_PREF"]==1, 0.5 + 0.5*df["topics_norm"], 0.0)  # 0..1

    df["si_norm"] = np.clip((df["SI_CONSIDERATION_num"] - 1)/2, 0, 1)  # S1=0, S2=0.5, S3=1
    df["tax_norm"] = np.clip((df["TAXONOMYPREF_num"] - 1)/2, 0, 1)      # A1=0, A2=0.5, A3=1

    return df

df_feat = encode(df_id)

plt.figure(figsize=(7,4))
plt.hist(df_feat["sfdr_gap"], bins=5)
plt.title("sfdr_gap distribution (clipped [-2,2])")
plt.xlabel("sfdr_gap")
plt.ylabel("count")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,4))
plt.hist(df_feat["sfdr_norm"], bins=10)
plt.title("sfdr_norm distribution (opportunity only, 0..1)")
plt.xlabel("sfdr_norm")
plt.ylabel("count")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

df_feat[["ID","si_offering","MIFID","SI_CONSIDERATION_num","si_norm","sfdr_gap","sfdr_norm","PAI_PREF","pai_block","TAXONOMYPREF_num","tax_norm"]].head()

---
## 6) Worked example (about 60%)

A stakeholder-friendly example under the **fixed rule**:
- MIFID = 1
- SI = S2 -> si_norm = 0.5
- Strong confirmations -> confirm = 1.0
- alpha = 0.80

Score = 100 * (0.8*0.5 + 0.2*1.0) = 60%

In [ ]:
example = pd.DataFrame([{
    "MIFID": 1,
    "si_norm": 0.5,
    "sfdr_norm": 1.0,
    "pai_block": 1.0,
    "tax_norm": 1.0
}])

alpha = 0.80
w = {"sfdr_norm":0.50, "pai_block":0.30, "tax_norm":0.20}
confirm = w["sfdr_norm"]*example["sfdr_norm"] + w["pai_block"]*example["pai_block"] + w["tax_norm"]*example["tax_norm"]
score = 100*(alpha*example["si_norm"] + (1-alpha)*confirm)

display(example.assign(confirm=confirm.round(3), score_percent=score.round(1)))

---
## 7) Train/Validation split

We evaluate all methods on the **same held-out validation** set.

In [ ]:
y = df_feat["si_offering"].astype(int).copy()
BASE_COLS = ["MIFID","si_norm","sfdr_norm","pai_block","tax_norm"]
X = df_feat[BASE_COLS].copy()

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)
idx_train, idx_val = X_train.index, X_val.index

print("Train:", len(idx_train), "Val:", len(idx_val))
print("Train si rate:", y_train.mean().round(4), "Val si rate:", y_val.mean().round(4))

---
## 8) Evaluation helpers (stakeholder metrics)

Primary operational metrics focus on the top bucket:
- Precision@K (top 10%, top 20%)
- Lift@K (top K positive rate / baseline rate)
- Decile lift curve
- Calibration curve

We also report AUC/AP/Brier for completeness.

In [ ]:
def eval_global(y_true, p, label):
    return {
        "model": label,
        "auc": roc_auc_score(y_true, p),
        "avg_precision": average_precision_score(y_true, p),
        "brier": brier_score_loss(y_true, p)
    }

def precision_lift_at_frac(y_true, p, frac=0.10):
    n = len(p)
    k = max(1, int(np.ceil(frac*n)))
    order = np.argsort(-p)
    top = y_true[order][:k]
    baseline = y_true.mean()
    prec = top.mean() if k > 0 else np.nan
    lift = (prec / baseline) if baseline > 0 else np.nan
    return {"frac": frac, "k": k, "precision": prec, "lift": lift, "baseline": baseline}

def lift_by_decile(y_true, p, n_bins=10):
    tmp = pd.DataFrame({"y": y_true, "p": p})
    tmp["decile"] = pd.qcut(tmp["p"], n_bins, labels=False, duplicates="drop") + 1
    out = tmp.groupby("decile")["y"].agg(["mean","count"]).rename(columns={"mean":"si_rate"})
    return out

def plot_lift_curve(tab, title):
    plt.figure(figsize=(8,4))
    plt.plot(tab.index, tab["si_rate"].values, marker="o")
    plt.title(title)
    plt.xlabel("Decile (1=lowest, 10=highest)")
    plt.ylabel("si_offering rate")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

def plot_calibration(y_true, p, title):
    prob_true, prob_pred = calibration_curve(y_true, p, n_bins=10, strategy="quantile")
    plt.figure(figsize=(6,6))
    plt.plot(prob_pred, prob_true, marker="o")
    plt.plot([0,1],[0,1], linestyle="--")
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("Observed rate")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

---
## 9) Method 1 — Fixed rule (baseline)

Fixed, stakeholder-friendly parameters:
- alpha = 0.80
- confirmation weights: SFDR 50%, PAI 30%, Taxonomy 20%

In [ ]:
@dataclass
class FixedCfg:
    alpha: float = 0.80
    w_sfdr: float = 0.50
    w_pai: float = 0.30
    w_tax: float = 0.20

cfg_fixed = FixedCfg()

def score_fixed(df: pd.DataFrame, cfg: FixedCfg) -> pd.DataFrame:
    df = df.copy()
    confirm = cfg.w_sfdr*df["sfdr_norm"] + cfg.w_pai*df["pai_block"] + cfg.w_tax*df["tax_norm"]
    score_m1 = cfg.alpha*df["si_norm"] + (1-cfg.alpha)*confirm
    score_m0 = df["si_norm"]
    p = np.where(df["MIFID"]==1, score_m1, score_m0)
    df["p_fixed"] = np.clip(p, 0, 1)

    # Explainability
    df["why_si"] = np.where(df["MIFID"]==1, cfg.alpha*df["si_norm"], df["si_norm"])
    df["why_sfdr"] = np.where(df["MIFID"]==1, (1-cfg.alpha)*cfg.w_sfdr*df["sfdr_norm"], 0.0)
    df["why_pai"]  = np.where(df["MIFID"]==1, (1-cfg.alpha)*cfg.w_pai*df["pai_block"], 0.0)
    df["why_tax"]  = np.where(df["MIFID"]==1, (1-cfg.alpha)*cfg.w_tax*df["tax_norm"], 0.0)
    return df

df_scored = score_fixed(df_feat, cfg_fixed)

p_fixed_val = df_scored.loc[idx_val, "p_fixed"].values
fixed_global = eval_global(y_val.values, p_fixed_val, "Fixed rule (alpha=0.80)")
fixed_top10 = precision_lift_at_frac(y_val.values, p_fixed_val, 0.10)
fixed_top20 = precision_lift_at_frac(y_val.values, p_fixed_val, 0.20)

display(pd.DataFrame([fixed_global]))
display(pd.DataFrame([fixed_top10, fixed_top20]))

tab = lift_by_decile(y_val.values, p_fixed_val)
display(tab)
plot_lift_curve(tab, "Lift by decile: Fixed rule")
plot_calibration(y_val.values, p_fixed_val, "Calibration: Fixed rule")

---
## 10) Method 2 — Weighted rule (recommended champion)

Same formula as fixed rule, but we learn:
1) confirmation weights (SFDR/PAI/Tax)
2) alpha split (bounded, conservative)

This keeps the score **rule-like**, explainable, and data-driven.

In [ ]:
# Learn confirmation weights on train subset where MIFID=1
train_df = df_scored.loc[idx_train].copy()
train_m1 = train_df[train_df["MIFID"]==1].copy()

CONF = ["sfdr_norm","pai_block","tax_norm"]

if train_m1["si_offering"].nunique() < 2:
    print("Warning: MIFID=1 train subset has one class; fallback to fixed weights.")
    w_learn = pd.Series([cfg_fixed.w_sfdr, cfg_fixed.w_pai, cfg_fixed.w_tax], index=CONF)
else:
    lr_w = LogisticRegression(max_iter=6000, class_weight="balanced")
    lr_w.fit(train_m1[CONF], train_m1["si_offering"])
    coef = pd.Series(lr_w.coef_[0], index=CONF)

    # Stakeholder-friendly monotonic constraint: keep non-negative contributions
    pos = np.maximum(coef.values, 0)
    if pos.sum() == 0:
        pos = np.ones_like(pos)
    w_learn = pd.Series(pos/pos.sum(), index=CONF)

display(w_learn.to_frame("learned_weight"))
plt.figure(figsize=(7,4))
plt.bar(w_learn.index, w_learn.values)
plt.title("Learned confirmation weights (sum=1)")
plt.ylabel("weight")
plt.grid(True, axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

# Choose alpha by CV on train (optimize Average Precision; bounded)
alpha_grid = np.round(np.arange(0.60, 0.91, 0.05), 2)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def score_prob(df: pd.DataFrame, alpha: float, w: pd.Series) -> np.ndarray:
    confirm = w["sfdr_norm"]*df["sfdr_norm"] + w["pai_block"]*df["pai_block"] + w["tax_norm"]*df["tax_norm"]
    m1 = alpha*df["si_norm"].values + (1-alpha)*confirm.values
    m0 = df["si_norm"].values
    return np.clip(np.where(df["MIFID"].values==1, m1, m0), 0, 1)

rows=[]
train_idx = idx_train.values
for a in alpha_grid:
    aps=[]
    for tr_i, te_i in skf.split(train_idx, y_train.values):
        te_ix = train_idx[te_i]
        df_te = df_scored.loc[te_ix]
        p = score_prob(df_te, a, w_learn)
        aps.append(average_precision_score(df_te["si_offering"].values, p))
    rows.append({"alpha": a, "cv_ap_mean": float(np.mean(aps))})

alpha_perf = pd.DataFrame(rows).sort_values("cv_ap_mean", ascending=False)
display(alpha_perf)

alpha_best = float(alpha_perf.iloc[0]["alpha"])
print("Selected alpha (max CV AP):", alpha_best)

plt.figure(figsize=(8,4))
tmp = alpha_perf.sort_values("alpha")
plt.plot(tmp["alpha"], tmp["cv_ap_mean"], marker="o")
plt.title("Selecting alpha by CV Average Precision (train)")
plt.xlabel("alpha (weight on SI)")
plt.ylabel("CV AP")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Apply weighted score
df_scored["p_weighted"] = score_prob(df_scored, alpha_best, w_learn)

# Explainability (why columns)
df_scored["whyW_si"] = np.where(df_scored["MIFID"]==1, alpha_best*df_scored["si_norm"], df_scored["si_norm"])
df_scored["whyW_sfdr"] = np.where(df_scored["MIFID"]==1, (1-alpha_best)*w_learn["sfdr_norm"]*df_scored["sfdr_norm"], 0.0)
df_scored["whyW_pai"]  = np.where(df_scored["MIFID"]==1, (1-alpha_best)*w_learn["pai_block"]*df_scored["pai_block"], 0.0)
df_scored["whyW_tax"]  = np.where(df_scored["MIFID"]==1, (1-alpha_best)*w_learn["tax_norm"]*df_scored["tax_norm"], 0.0)

p_w_val = df_scored.loc[idx_val, "p_weighted"].values
w_global = eval_global(y_val.values, p_w_val, f"Weighted rule (alpha={alpha_best})")
w_top10 = precision_lift_at_frac(y_val.values, p_w_val, 0.10)
w_top20 = precision_lift_at_frac(y_val.values, p_w_val, 0.20)

display(pd.DataFrame([w_global]))
display(pd.DataFrame([w_top10, w_top20]))

tab = lift_by_decile(y_val.values, p_w_val)
display(tab)
plot_lift_curve(tab, "Lift by decile: Weighted rule (champion)")
plot_calibration(y_val.values, p_w_val, "Calibration: Weighted rule (champion)")

---
## 11) Method 3 — ML (Calibrated Logistic Regression) — challenger

ML is run in shadow mode until we have pilot labels.

To respect the business structure, we use MiFID interaction features, so confirmations only matter when `MIFID=1`.

In [ ]:
def ml_matrix(df: pd.DataFrame) -> pd.DataFrame:
    X = pd.DataFrame(index=df.index)
    X["si_norm"] = df["si_norm"]
    X["MIFID"] = df["MIFID"]
    X["m1_sfdr"] = df["MIFID"] * df["sfdr_norm"]
    X["m1_pai"]  = df["MIFID"] * df["pai_block"]
    X["m1_tax"]  = df["MIFID"] * df["tax_norm"]
    return X

Xtr = ml_matrix(df_scored.loc[idx_train])
Xva = ml_matrix(df_scored.loc[idx_val])

base_lr = LogisticRegression(max_iter=8000, class_weight="balanced")
cal_lr = CalibratedClassifierCV(base_lr, method="isotonic", cv=5)
cal_lr.fit(Xtr, y_train)

p_ml_val = cal_lr.predict_proba(Xva)[:,1]
ml_global = eval_global(y_val.values, p_ml_val, "ML: Calibrated LR")
ml_top10 = precision_lift_at_frac(y_val.values, p_ml_val, 0.10)
ml_top20 = precision_lift_at_frac(y_val.values, p_ml_val, 0.20)

display(pd.DataFrame([ml_global]))
display(pd.DataFrame([ml_top10, ml_top20]))

tab = lift_by_decile(y_val.values, p_ml_val)
display(tab)
plot_lift_curve(tab, "Lift by decile: ML (calibrated LR)")
plot_calibration(y_val.values, p_ml_val, "Calibration: ML (calibrated LR)")

# Interpretability: coefficients from an uncalibrated LR fitted on same features
plain_lr = LogisticRegression(max_iter=8000, class_weight="balanced")
plain_lr.fit(Xtr, y_train)
coef = pd.Series(plain_lr.coef_[0], index=Xtr.columns).sort_values(key=np.abs, ascending=False)
display(coef.to_frame("coef"))

plt.figure(figsize=(9,4))
plt.bar(coef.index[:10], coef.values[:10])
plt.title("Top coefficients (uncalibrated LR; sign indicates direction)")
plt.ylabel("coef")
plt.xticks(rotation=45, ha="right")
plt.grid(True, axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

---
## 12) Compare all methods on validation

We choose the champion based on top-bucket lift/precision and stability.

Recommendation: use the Weighted rule as the default unless ML is materially better and stable over time.

In [ ]:
comparison = pd.DataFrame([fixed_global, w_global, ml_global]).round(4)
display(comparison)

topk = pd.DataFrame([
    {"model": "Fixed", **fixed_top10},
    {"model": "Fixed", **fixed_top20},
    {"model": "Weighted", **w_top10},
    {"model": "Weighted", **w_top20},
    {"model": "ML", **ml_top10},
    {"model": "ML", **ml_top20},
]).round(4)
display(topk)

for metric in ["auc","avg_precision","brier"]:
    plt.figure(figsize=(8,4))
    plt.bar(comparison["model"], comparison[metric])
    plt.title(f"Validation comparison: {metric}")
    plt.ylabel(metric)
    plt.xticks(rotation=20, ha="right")
    plt.grid(True, axis="y", alpha=0.3)
    plt.tight_layout()
    plt.show()

---
## 13) Operational output

Output format:
- rank clients with `si_offering=0`
- add percentiles + buckets (Low/Average/High)
- include why columns so the list is explainable

Default: use the Weighted rule probability `p_weighted`.

In [ ]:
df_out = df_scored.copy()
df_out["rank_prob"] = df_out["p_weighted"]
df_out["score_percentile"] = (df_out["rank_prob"].rank(pct=True) * 100).round(2)
df_out["bucket_3"] = pd.cut(df_out["score_percentile"], bins=[-0.01, 50, 80, 100], labels=["Low","Average","High"])

targets = df_out[df_out["si_offering"]==0].sort_values("rank_prob", ascending=False).head(20)

cols = [
    "ID","rank_prob","score_percentile","bucket_3",
    "MIFID","SI_CONSIDERATION_num","sfdr_gap","PAI_PREF","TAXONOMYPREF_num","esg_topics_yes_cnt",
    "whyW_si","whyW_sfdr","whyW_pai","whyW_tax"
]
targets[cols]

---
## 14) Risks & governance

### Key risks
- Label bias: proxy membership is not true interest
- Leakage risk: never use `OFFERING_NAME` as a feature (we only use it to build the proxy label)
- Drift: questionnaire completion, product naming, and advisory behavior change over time
- Missing data: defaulting to lowest tier can under-score clients

### Controls
- Monitor monthly: missingness, score distributions, top-bucket hit rate
- Refit: confirmation weights + alpha quarterly (or after process change)
- Keep a human review step for the highest-ranked edge cases
- Use ML only if it stays calibrated and stable

### Why not kNN/SVM here
- Harder explainability and operational calibration
- More sensitivity to scaling and missingness
- Higher overfit risk with proxy labels
Calibrated LR is usually the best trade-off for stakeholder trust + performance in this setting.

---
## 15) Pilot plan (to create true labels)

To make this truly optimal, we need true outcomes.

### Pilot design
- Population: `si_offering=0`
- Treatment: top bucket (e.g., top 20% by weighted rule)
- Control: randomized sample from remaining eligible IDs
- Outcomes: response, meeting booked, adoption / mandate change, pipeline created

### Next modeling step (post-pilot)
Train on true outcomes and consider uplift modeling (incremental impact of outreach), which is the real ROI-optimal objective.